In [226]:
def find(s, ch):
    return [i for i, ltr in enumerate(s) if ltr == ch]

def find_all(a_str, sub):
    start = 0
    while True:
        start = a_str.find(sub, start)
        if start == -1: return
        yield start
        start += len(sub)

In [227]:
import csv

# initializing the titles and rows list 
fields = [] 
rows = [] 
# raw_questions = []
raw_questions_to_hw = dict()

filename = 'contributions.csv'
# reading csv file 
with open(filename, 'r', encoding="utf8") as csvfile: 
    # creating a csv reader object 
    csvreader = csv.reader(csvfile) 
      
    # extracting field names through first row 
    fields = next(csvreader) 
  
    # extracting each data row one by one 
    for row in csvreader: 
        rows.append(row) 

    curr_question = ""
    curr_text = ""
    curr_topic = ""
    curr_folder = ""
    curr_answer = ""
    curr_number = 0
    curr_follow_up = 0
    endorsed = 0
    
    # This will help us take the most recent version because it always comes first
    already_filled = 0
    for row in rows[:]: 
        # parsing each column of a row 
        for col,field in zip(row, fields): 
            if(field == "Post Number" and curr_number != int(col)):
                curr_folders = curr_folder.split()
                for folder in curr_folders:
                    if folder[-1] == ';':
                            folder = folder[:-1]
                    if(curr_question != "" and curr_answer != ""):
                        if folder in raw_questions_to_hw:
                            raw_questions = raw_questions_to_hw[folder]
                            raw_questions.append(curr_question)
                            raw_questions_to_hw.update({folder : raw_questions})
                        else:
                            raw_questions = []
                            raw_questions.append(curr_question)
                            raw_questions_to_hw.update({curr_folder : raw_questions})
                curr_number = int(col)
                curr_follow_up = 0
                curr_question = ""
                already_filled = 0
                curr_answer = ""
                curr_topic = ""
                curr_folder = ""
                endorsed = 0
                
            elif(field == "Submission HTML Removed"): #the question could also be in subject
                curr_text = col
                
            elif(field == "Subject"): #We want to check if this is actually the question eventaully
                curr_topic = col
                
            elif(field == "Folders"):
                curr_folder = col
                
            elif(field == "Endorsed by Instructor"):
                if(col == "TRUE"):
                    endorsed = 1
                else:
                    endorsed = 0
            elif(field == "Part of Post"):
                if(col == "started_off_question"):
                    if(already_filled == 0):
                        curr_question = curr_text
                    already_filled = 0
                elif(col == "updated_question"):
                    # update question entry to post and follow up numbers
                    if(already_filled == 0):
                        curr_question = curr_text
                        already_filled = 1
                        
                elif(col == "updated_i_answer"):
                    # update last entry to the post and follow up number
                    # what if it was appended? how do we update just the end?
                    if(already_filled == 0):
                        curr_answer = curr_answer + "\n" + curr_text
                        already_filled = 1
                    endorsed = 1
                elif(col == "started_off_i_answer"):
                    if(already_filled == 0):
                        curr_answer = curr_answer + "\n" + curr_text
                    already_filled = 0
                    endorsed = 1
                    
                elif(col == "updated_s_answer"):
                    # update last entry to the post and follow up number
                    # what if it was appended? how do we update just the end?
                    if(already_filled == 0):
                        curr_answer = curr_answer + "\n" + curr_text
                        already_filled = 1
                elif(col == "started_off_s_answer"):
                    # append to answer - if it was previously endorsed and were appending should we keep it endorsed?
                    if(already_filled == 0):
                        curr_answer = curr_answer + "\n" + curr_text
                    already_filled = 0
                    
                elif(col == "followup"):
                    # new entry for the new followup question 
                    curr_folders = curr_folder.split()
                    for folder in curr_folders:
                        if folder[-1] == ';':
                            folder = folder[:-1]
                        if folder in raw_questions_to_hw:
                            raw_questions = raw_questions_to_hw[folder]
                            raw_questions.append(curr_question)
                            raw_questions_to_hw.update({folder : raw_questions})
                        else:
                            raw_questions = []
                            raw_questions.append(curr_question)
                            raw_questions_to_hw.update({folder : raw_questions})
                    curr_question = ""
                    curr_answer = ""
                    curr_folder = ""
                    endorsed = 0
                    already_filled = 0
                        
                    curr_follow_up = curr_follow_up + 1
                    curr_question = curr_text
                elif(col == "reply_to_followup"):
                    curr_answer = curr_answer + "\n" + curr_text
                    
num = 0
# for i in raw_questions_to_hw["hw5"]:
#     print(num)
#     print(i)
#     print('\n')
#     num = num+1

# myDB.close()


@92 are you using safari?
92

When thinking about this remember that (at least for now) everything in Ocaml simplifies down to one expression. So the whole then section needs to be able to evaluate to 1 final expression, hence you cannot really put 2 expressions that you want to end up with into one then statement.Rather than thinking about the if-then as DOING things, think of it as becoming something.
This may also help @23

23

@14
1

I took a look at your code and it seems that re-building and re-running it seemed to work and produced no console errors.  Try that and let us know if that fixes anything.

Edit: I think the error highlighting was a little slow - closing and reopening your project seemed to fix that. @33
3

@89 @44
89

@89 @44
4

@44 for your second question and for your first, don't include the negative readings in the final answer.

e.g. 1,2,3,-1,-2,-3,-999 should have the average 2. 
44

For debugging purposes, I'd strongly suggest using test cases to help you figu

IndexError: string index out of range

In [228]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from sklearn.metrics import adjusted_rand_score

from xml.dom import minidom

folder = "hw2"
print(len(raw_questions_to_hw[folder]))

# parse an xml file by name
mydoc = minidom.parse('Posts.xml')

SEitems = mydoc.getElementsByTagName('row')

SEitems_questions = []
for item in SEitems:
    SEitems_questions.append(item.attributes['Body'].value)
    
SEitems_questions = SEitems_questions[1:6000]

vectorizer = TfidfVectorizer(stop_words='english')
X_orig = vectorizer.fit(SEitems_questions)
X = vectorizer.transform(raw_questions_to_hw[folder])

# Number of clusters per homework
true_k = 50
model = KMeans(n_clusters=true_k, init='k-means++', max_iter=100, n_init=10)
model.fit(X)

# print("\n")
# print("Prediction")



302


KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=100,
    n_clusters=50, n_init=10, n_jobs=None, precompute_distances='auto',
    random_state=None, tol=0.0001, verbose=0)

In [229]:
# Combine clusters that have references to each other

Y = vectorizer.transform(["Should we fail for empty queues for delete and contains too?"])
prediction = model.predict(Y)
# print(prediction)

# for x in range(276):
#     if model.labels_[x] == 28:
#         print("Question Number: ") 
#         print(x)
#         print(raw_questions_to_hw[folder][x])

In [230]:
# Create a list of lists for each cluser

final_clusters = {}
for i in range(len(raw_questions_to_hw[folder])):
    label = model.labels_[i]
    final_clusters.setdefault(label, []).append(raw_questions_to_hw[folder][i])
    
print(raw_references)
    
# print(final_clusters)

{22: [92, 150, 229], 92: [22, 98, 190, 933], 27: [23, 274, 274, 349, 355], 23: [27, 296, 410, 447], 30: [1, 326, 411], 1: [30, 473], 35: [3, 383, 416], 3: [35], 37: [89, 4], 89: [37, 977], 4: [37, 64, 128, 159, 544], 46: [44, 159], 44: [46], 49: [47, 120, 575], 47: [49, 128], 52: [38, 57, 537, 591, 654], 38: [52, 57, 157, 405], 57: [52, 38, 72, 618, 646, 778], 64: [4, 159, 641], 68: [18], 18: [68, 223, 465, 465, 465, 465, 465, 465, 465, 465, 465, 465, 694, 695], 72: [32, 57], 32: [72, 356, 605, 727], 79: [5], 5: [79, 181, 341], 80: [40, 903, 935], 40: [80], 83: [-1, -1, 870], -1: [83, 83, 99, 99, 108, 123, 123, 135, 172, 213, 251, 283, 283, 310, 324, 327, 477, 478, 499, 594, 594, 790, 883, 923, 923, 976], 98: [92, 189], 99: [-1, -1], 108: [-1], 113: [69], 69: [113], 116: [100], 100: [116], 120: [49], 123: [-1, -1], 128: [47, 4], 135: [-1], 140: [8], 8: [140], 143: [14], 14: [143, 200, 242], 146: [16], 16: [146, 197, 316], 150: [22], 157: [38], 159: [106, 101, 46, 64, 4], 106: [159], 10

In [221]:
# Shorten the sentences just for one cluster for now
import random
from nltk.tokenize import word_tokenize
import string

number, items = random.choice(list(final_clusters.items()))

stop_words = ["ourselves", "hers", "between", "yourself", "but", "again", 
                  "there", "about", "once", "during", "out", "very", "having", 
                  "with", "they", "own", "an", "be", "some", "for", "do", "its", 
                  "yours", "such", "into", "of", "most", "itself", "other", "off",
                  "is", "s", "am", "or", "who", "as", "from", "him", "each", "the",
                  "themselves", "until", "below", "are", "we", "these", "your", "his", 
                  "through", "don", "nor", "me", "were", "her", "more", "himself", "this",
                  "down", "should", "our", "their", "while", "above", "both", "up", "to",
                  "ours", "had", "she", "all", "no", "when", "at", "any", "before", "them",
                  "same", "and", "been", "have", "in", "will", "on", "does", "yourselves", 
                  "then", "that", "because", "what", "over", "why", "so", "can", "did", 
                  "not", "now", "under", "he", "you", "herself", "has", "just",
                  "where", "too", "only", "myself", "which", "those", "i", "after",
                  "few", "whom", "t", "being", "if", "theirs", "my", "against", "a",
                  "by", "doing", "it", "how", "further", "was", "here", "than", "ok",
                  "thank", "ah"]

length = len(items)
shortened_questions = []
for i in range(0, length):
    question = items[i]
    tokens = word_tokenize(question)
    tokens = [w.lower() for w in tokens]
    words = [w for w in tokens if (not w in stop_words and w != "" and w != " ")]

    # Get rid of punctuation
    table = str.maketrans('', '', string.punctuation)
    words = [w.translate(table) for w in words]

    final_question = " ".join(words)

    shortened_questions.append(final_question)

#     shortened_questions = [s for s in shortened_questions if s != '']

#     # Printing out average
# totalNumWords = 0
# maxLength = 0
# for s in shortened_questions:
#     numWords = len(word_tokenize(s))
#     if (maxLength < numWords): 
#         maxLength = numWords
#     # print(numWords)
#     if (numWords > 50):
#         print(s)
#         print("\n")
#     totalNumWords += numWords

print(shortened_questions)



['function  return something one nucleotides  c  g   like  na  error  s one test cases want make sure ', 'test see actually fails  ca nt really set boolean equal fail ', 'hw2  due stringency grading test cases create catch branch pattern match  something like      gt  somethingx  tl   gt  somthing  tl   gt  failwith  invalid input  also  last catchall branch necessary  write test case ', 'write test case something outputs failwith', 'added test cases hammingdistance  one given appeared run others nt appear run project window  wrote way test cases written could problem ', 'written additional test cases acidsofhelix function  however showing terminal build run program  tried refreshing codio  anything resolve  edit  resolved  forgot write  test  end   runtest', 'wrote couple test cases helper function dnaml  nt showing run project  syntax one cases  let test    bool  endofacidchain   runtest  endofacidchain correct stop codon  test m sure s wrong ', 'kind test cases create test decreasin

In [222]:
from bert_serving.client import BertClient 

# Start BERT client for mrpc bert-serving-start -model_dir /Users/maralevy/Downloads/uncased_L-12_H-768_A-12 -tuned_model_dir /Users/maralevy/Documents/Piazza-SeniorDesign/bert/mrpc_output -num_worker=4 -max_seq_len=50
# Start BERT client for qqa checkpoint bert-serving-start -model_dir /Users/maralevy/Downloads/uncased_L-12_H-768_A-12 -tuned_model_dir /Users/maralevy/Documents/Piazza-SeniorDesign/bert/qqp_output -num_worker=4 -max_seq_len=50 -ckpt_name bert_model.ckpt-1000
bc = BertClient(check_version=False)  
doc_vecs = bc.encode(shortened_questions)

In [223]:
# This cell allows you to type a question and find semantically similar questions using BURT, eventually
# we should run this for every question in raw_questions

import np
print(doc_vecs)
topk = len(shortened_questions)
final_scores = {}
for question in shortened_questions:
    query_vec = bc.encode([question])
    # compute simple dot product as score
    score = np.sum(query_vec * doc_vecs, axis=1)/np.linalg.norm(doc_vecs, axis=1)
    topk_idx = np.argsort(score)[::-1][:topk]
#     print("Question: ")
#     print(question)
    for idx in topk_idx:
        if idx != topk_idx[0]:
            final_scores.setdefault(items[topk_idx[0]], []).append((score[idx], items[idx]))

print(final_scores)

[[-0.86522406 -0.01704413  0.26169822 ... -0.00269519 -0.694843
   0.8898976 ]
 [-0.9710178   0.2253393   0.19439118 ...  0.14522651 -0.82292557
   0.454266  ]
 [-1.1648377   0.18451458  0.3001394  ...  0.22001126 -0.8783203
   0.70074236]
 ...
 [-1.2101018   0.7623831   0.15583    ... -0.31735602 -0.3317412
   0.29978138]
 [-0.37461233  0.20903824  0.18264975 ...  0.10436659 -0.5598015
   0.10364687]
 [-1.0511183  -0.07000171  0.23072939 ...  0.12005223 -1.0738326
   0.65582544]]
{'For this function, should we return something if one of the nucleotides is not A,C,G, or T? Like "N/A" or an error? It\'s not one of the test cases so just want to make sure.\xa0': [(18.93522, 'What kind of test cases should we create to test decreasing_similarity?\n\nAre we supposed to add another type of\xa0question_most_like_human function and change the order of the helix output?'), (18.901247, "Are these test cases right for the count_leaves function? My test cases all pass, but when I submitted the ho

In [224]:
cluster_number = 0
final_clusters = {}
used_strings = {}
for key in final_scores:
    quota = float(final_scores[final_scores[key][0][1]][0][0] - final_scores[final_scores[key][0][1]][1][0])
    if key == final_scores[final_scores[key][0][1]][0][1] or \
    (key == final_scores[final_scores[key][0][1]][1][1] and \
     (abs(final_scores[final_scores[key][0][1]][1][0] - final_scores[final_scores[key][0][1]][0][0]) < quota * 2)) or \
    (key == final_scores[final_scores[key][0][1]][2][1] and \
     (abs(final_scores[final_scores[key][0][1]][2][0] - final_scores[final_scores[key][0][1]][0][0]) < quota * 3)):        
        # Combine clusters if needed
        if key in used_strings and final_scores[key][0][1] in used_strings:
            cluster_number_one = used_strings[key]
            cluster_number_two = used_strings[final_scores[key][0][1]]
            if cluster_number_two != cluster_number_one:
                final_clusters[cluster_number_one] = final_clusters[cluster_number_one] + final_clusters[cluster_number_two]
                for string in final_clusters[cluster_number_two]:
                    used_strings[string] = cluster_number_one
                final_clusters.pop(cluster_number_two, None)
        elif key in used_strings:
            final_clusters[used_strings[key]].append(final_scores[key][0][1])
        elif final_scores[key][0][1] in used_strings:
            final_clusters[used_strings[final_scores[key][0][1]]].append(key)
        else:
            used_strings[key] = cluster_number
            used_strings[final_scores[key][0][1]] = cluster_number
            final_clusters[cluster_number] = [key, final_scores[key][0][1]]
            cluster_number = cluster_number + 1
            
for key in final_clusters:
    print("Cluster %d:"% key)
    for q in final_clusters[key]:
        print("Q: \n %s" % q)
        


Cluster 0:
Q: 
 How do we test to see if it actually fails? We can't really set a boolean to equal fail. 
Q: 
 For some reason when I run setList in order to test the functions I wrote for listSet, setList fails to compile and I'm not sure why.
Cluster 2:
Q: 
 When I added more test cases for hamming_distance, only the one that was given appeared to run and the others don't appear in the Run Project window. I wrote them in the same way all the test cases are written so what could be the problem?
Q: 
 Are these test cases right for the count_leaves function? My test cases all pass, but when I submitted the homework I had 4 failures on Problem 4 (and this is the only function we had to write in it), so I'm concerned that my understanding of the function is not entirely correct.


Q: 
 In hw2, due to the stringency in grading test cases do we have to create a catch all branch during pattern match? Something like this
|[] -&gt; do something|x::tl -&gt; do somthing|_::tl -&gt; failwith "Inv